In [72]:
import numpy as np
import pandas as pd

from datetime import datetime

from pathlib import Path

import talib

from math import *

In [2]:
class CSVDataHandler:
    def __init__(self, symbol_list, timeframe):
        self.csv_dir = '../exchange_data'
        self.symbol_list = ['-'.join(symbol.split('/'))
                            for symbol in symbol_list]
        self.timeframe = timeframe

        self.symbol_data = dict()
        self.latest_symbol_data = dict()

        self.continue_backtest = True

        self._load_symbol_data()

    def __repr__(self):
        return f'<CSVDataHandler>'

    def __str__(self):
        return f'CSVDataHandler with {self.timeframe} timeframe'

    def _load_symbol_data(self):
        '''
        It imports historical data from a set of CSV files then prepare them for the backtesting.
        It sets self.symbol_data with all data and init self_latest_symbol_data with empty data.
        '''
        csv_files_path = f'{Path().absolute()}/{self.csv_dir}'
        columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        combined_symbol_index = None

        for symbol in self.symbol_list:
            symbol_csv_path = f'{csv_files_path}/{symbol}_{self.timeframe}.csv'
            self.symbol_data[symbol] = pd.read_csv(
                symbol_csv_path, header=None, index_col=0, names=columns)

            self.symbol_data[symbol].index = pd.to_datetime(
                self.symbol_data[symbol].index, unit='ms')
            self.symbol_data[symbol] = self.symbol_data[symbol].drop_duplicates(
            )
            self.symbol_data[symbol] = self.symbol_data[symbol].sort_index()

            if combined_symbol_index is None:
                combined_symbol_index = self.symbol_data[symbol].index
            else:
                combined_symbol_index = combined_symbol_index.union(
                    self.symbol_data[symbol].index)

            # Init list that will contain latest ohlcv data for each symbol
            self.latest_symbol_data[symbol] = list()

        for symbol in self.symbol_list:
            self.symbol_data[symbol] = self.symbol_data[symbol].reindex(
                index=combined_symbol_index, method='pad', fill_value=0).itertuples(name='OHLCV')

    def _get_new_bar(self, symbol):
        for bar in self.symbol_data[symbol]:
            yield bar

    def update_bars(self):
        """
        It feeds the backtest with latest data in each iteration and trigger Market event.
        It sets this data in self.latest_symbol_data.
        If there is no data to feed the backtest, it raises an exception and the backtest stops.
        """
        for symbol in self.symbol_list:
            try:
                bar = next(self._get_new_bar(symbol))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[symbol].append(bar)

    def get_latest_bar(self, symbol):
        """
        It returns the latest bar data of a given symbol as namedtuple
        e.g: OHLCV(Index=Timestamp('2017-08-08 11:00:00'), open=0.080292,
                   high=0.081039, low=0.079854, close=0.080282, volume=610.868)
        """
        try:
            return self.latest_symbol_data[symbol][-1]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise

    def get_latest_bars(self, symbol, N=1):
        """
        It returns the latest bars data of a given symbol as a np array
        """
        try:
            return np.array(self.latest_symbol_data[symbol][-N:])
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars_df(self, symbol, N=1):
        """
        It returns the latest bars data of a given symbol as a np array
        """
        try:
            bars = pd.DataFrame(self.latest_symbol_data[symbol][-N:])
            bars.set_index('Index', inplace=True)
            return bars
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise

    def get_latest_bar_datetime(self, symbol):
        """
        It returns the latest bar timestamp object of a given symbol
        """
        try:
            return self.latest_symbol_data[symbol][-1].Index
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise

    def get_latest_bar_value(self, symbol, value_type):
        """
        It returns the latest bar value (open, high, low, close or volume) of a given symbol
        """
        try:
            if value_type == 'datetime':
                return getattr(self.latest_symbol_data[symbol][-1], 'Index')
            return getattr(self.latest_symbol_data[symbol][-1], value_type)
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise

    def get_latest_bars_values(self, symbol, value_type, N=1):
        """
        It returns a numpy array of the latest bars values (open, high, low, close or volume) of a given symbol
        e.g: array([ 89, 88.2, 86.4, ...])
        """
        try:
            bars = self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
        else:
            if value_type == 'datetime':
                return np.array([getattr(bar, 'Index') for bar in bars])
            return np.array([getattr(bar, value_type) for bar in bars])

    def current_price(self, symbol):
        """
        It retuns latest close price
        """
        return self.get_latest_bar_value(symbol, 'close')


In [5]:
data = CSVDataHandler(['BTC/USDT'], '1h')

In [6]:
for _ in range(250):
    data.update_bars()

bars = data.get_latest_bars_df('BTC-USDT', N=250)

In [7]:
bars['rsi'] = talib.RSI(bars['close'])

In [8]:
price_set = bars.loc[bars['open'] > bars['close'], 'low']
rsi_set = bars.loc[bars['open'] > bars['close'], 'rsi']

In [9]:
bars

,open,high,low,close,volume,rsi
Index,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,NaN
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,NaN
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,NaN
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,NaN
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,NaN
...,...,...,...,...,...,...
2017-08-27 09:00:00,4323.90,4345.81,4302.11,4320.81,12.868744,51.750829
2017-08-27 10:00:00,4330.00,4330.00,4285.56,4302.73,13.820714,48.092588
2017-08-27 11:00:00,4323.87,4344.99,4285.80,4323.37,14.332417,52.242971


In [10]:
rsi_set

Index
2017-08-17 06:00:00          NaN
2017-08-17 11:00:00          NaN
2017-08-17 12:00:00          NaN
2017-08-17 15:00:00          NaN
2017-08-17 17:00:00          NaN
                         ...    
2017-08-27 09:00:00    51.750829
2017-08-27 10:00:00    48.092588
2017-08-27 11:00:00    52.242971
2017-08-27 12:00:00    47.360671
2017-08-27 13:00:00    50.206067
Name: rsi, Length: 124, dtype: float64

In [89]:
trades = pd.read_csv('../backtest_results/trades.csv', header=0, parse_dates=True, index_col=0)

In [106]:
start_date = '2019-01-14 00:00:00'
end_date = '2019-07-02 00:00:00'
mask = (trades['open_date'] >= start_date) & (trades['open_date'] <= end_date)
trades_2017 = trades.loc[mask]

In [107]:
records = trades_2017[['open_date', 'open_market_price', 'close_date', 'close_market_price', 'win_trades_long', 'loss_trades_long', 'returns_long']]

In [108]:
records

,open_date,open_market_price,close_date,close_market_price,win_trades_long,loss_trades_long,returns_long
332,2019-01-15 15:00:00,3602.33,2019-01-16 04:00:00,3578.36,False,True,-0.006654
333,2019-01-18 17:45:00,3577.44,2019-01-19 02:15:00,3607.59,True,False,0.008428
334,2019-01-23 14:15:00,3570.84,2019-01-24 07:30:00,3548.22,False,True,-0.006335
335,2019-01-28 04:45:00,3479.01,2019-01-28 22:30:00,3433.75,False,True,-0.013009
336,2019-02-06 10:00:00,3406.30,2019-02-06 15:30:00,3415.34,True,False,0.002654
337,2019-02-10 14:45:00,3619.50,2019-02-10 16:30:00,3642.20,True,False,0.006272
338,2019-02-24 12:15:00,4137.61,2019-02-25 02:45:00,3799.91,False,True,-0.081617
339,2019-02-27 00:00:00,3806.03,2019-02-27 02:30:00,3816.07,True,False,0.002638
340,2019-03-04 08:15:00,3725.95,2019-03-04 16:00:00,3725.14,False,True,-0.000217
341,2019-03-11 11:15:00,3877.23,2019-03-11 18:15:00,3880.94,True,False,0.000957


In [109]:
print(records['win_trades_long'].sum()/records['loss_trades_long'].sum())

4.625


In [86]:
n = 2.3
m = 2.5

In [87]:
ceil(n)

3

In [105]:
records['returns_long'].sum()

0.5210572881722365